Data Class

In [96]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"
    NEUTRAL = "NEUTRAL"

class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
    
    def get_sentiment(self):
        if(self.score <= 2):
            return Sentiment.NEGATIVE
        elif(self.score <= 3):
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))

        positive_shunk = positive[:len(negative)]
        self.reviews = negative + positive_shunk
        random.shuffle(self.reviews)
        

Data Loading

In [97]:
import json
file_name = 'Books_small_10000.jsonl'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

print(reviews[5].score)
print(reviews[5].text)
print(reviews[5].sentiment)


5.0
I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.
POSITIVE


Data Prep

In [98]:
from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews,test_size=0.33,random_state=42)

train_container = ReviewContainer(training)
train_container.evenly_distribute()

test_container = ReviewContainer(test)
test_container.evenly_distribute()

In [99]:
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


Bag of words vectorization

In [100]:
from  sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

train_x_vector = vectorizer.fit_transform(train_x)

test_x_vector = vectorizer.transform(test_x)

print(vectorizer.get_feature_names_out())
print(train_x[0])
print(train_x_vector[0].toarray())


['00' '000' '01' ... 'zombies' 'zone' 'zora']
I loved the first two and read them in a day each.  I struggled to get through this one.  As every one mentioned, the dual character perspective is annoying and impossible to tell which character is speaking since they are written in exactly the same voice.  Many people also mentioned that the ending was sad.  Without giving away what happened, I will say that it did not make me remotely sad because I was no longer interested in the characters by that point.  The first two books were great because although they had great character development, they were really plot focused.  This one just tries to explain why everything is the way it is without having a really interesting plot.  Snoozefest.  It makes me rethink hating the &#34;Lost&#34; (tv series) ending.  I hated that because they never explained anything but maybe some things are just better left to the imagination.  It's so disappointing because I LOVED the first book and really enjoyed

Classification

Support Vector Machine(SVM)

In [101]:
from sklearn import svm

clf_svm = svm.SVC(kernel="linear")

clf_svm.fit(train_x_vector,train_y)

clf_svm.predict(test_x_vector[0])


array(['NEGATIVE'], dtype='<U8')

Decision Tree

In [102]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(train_x_vector,train_y)

clf_dec.predict(test_x_vector[0])

array(['NEGATIVE'], dtype='<U8')

Naive Bayes

In [103]:
from sklearn.naive_bayes import GaussianNB

clf_nb = GaussianNB()

clf_nb.fit(train_x_vector.toarray(),train_y)

clf_nb.predict(test_x_vector[0].toarray().reshape(1,-1))

array(['POSITIVE'], dtype='<U8')

Logistic Regression

In [104]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(train_x_vector,train_y)

clf_log.predict(train_x_vector[0])

array(['NEGATIVE'], dtype='<U8')

Evaluation

In [105]:
# Mean Accurancy
print(clf_svm.score(test_x_vector,test_y))
print(clf_dec.score(test_x_vector,test_y))
print(clf_nb.score(test_x_vector.toarray(), test_y))
print(clf_log.score(test_x_vector,test_y))


0.7980769230769231
0.6418269230769231
0.6346153846153846
0.8173076923076923


In [108]:
# F1 Score
from sklearn.metrics import f1_score

print(f1_score(test_y,clf_svm.predict(test_x_vector),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y,clf_dec.predict(test_x_vector),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y,clf_nb.predict(test_x_vector.toarray()),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))
print(f1_score(test_y,clf_log.predict(test_x_vector),average=None,labels=[Sentiment.POSITIVE,Sentiment.NEGATIVE]))

[0.8028169  0.79310345]
[0.64096386 0.64268585]
[0.59574468 0.66666667]
[0.82325581 0.81094527]


In [113]:
test_set = ['Very good', "bad book do not buy", 'fucking good']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'POSITIVE'], dtype='<U8')